In [2]:
import argparse
import cv2
import numpy as np
import os
import torch
import pdb

from utils import setup_seed, read_points, read_calib, read_label, \
    keep_bbox_from_image_range, keep_bbox_from_lidar_range, vis_pc, \
    vis_img_3d, bbox3d2corners_camera, points_camera2image, \
    bbox_camera2lidar

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def point_range_filter(pts, point_range=[0, -39.68, -3, 69.12, 39.68, 1]):
    '''
    data_dict: dict(pts, gt_bboxes_3d, gt_labels, gt_names, difficulty)
    point_range: [x1, y1, z1, x2, y2, z2]
    논문에 표기된 대로 car, pedstrian cyclist를 검출하는 포인터 범위
    '''
    flag_x_low = pts[:, 0] > point_range[0] # 0
    flag_y_low = pts[:, 1] > point_range[1] # -39.68
    flag_z_low = pts[:, 2] > point_range[2] # -3
    flag_x_high = pts[:, 0] < point_range[3] # 69.12
    flag_y_high = pts[:, 1] < point_range[4] # 39.68
    flag_z_high = pts[:, 2] < point_range[5] # 1
    keep_mask = flag_x_low & flag_y_low & flag_z_low & flag_x_high & flag_y_high & flag_z_high
    pts = pts[keep_mask]
    return pts 

In [3]:
import numpy as np
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
from model.anchors import Anchors, anchor_target, anchors2bboxes
from ops import Voxelization, nms_cuda
from utils import limit_period


class PillarLayer(nn.Module):
    def __init__(self, voxel_size, point_cloud_range, max_num_points, max_voxels):
        super().__init__()
        self.voxel_layer = Voxelization(voxel_size=voxel_size,
                                        point_cloud_range=point_cloud_range,
                                        max_num_points=max_num_points,
                                        max_voxels=max_voxels)

    @torch.no_grad()
    def forward(self, batched_pts):
        '''
        batched_pts: list[tensor], len(batched_pts) = bs
        return: 
               pillars: (p1 + p2 + ... + pb, num_points, c), 
               coors_batch: (p1 + p2 + ... + pb, 1 + 3), 
               num_points_per_pillar: (p1 + p2 + ... + pb, ), (b: batch size)
        '''
        pillars, coors, npoints_per_pillar = [], [], []
        for i, pts in enumerate(batched_pts):
            voxels_out, coors_out, num_points_per_voxel_out = self.voxel_layer(pts) 
            # voxels_out: (max_voxel, num_points, c), coors_out: (max_voxel, 3)
            # num_points_per_voxel_out: (max_voxel, )
            pillars.append(voxels_out)
            coors.append(coors_out.long())
            npoints_per_pillar.append(num_points_per_voxel_out)
        
        pillars = torch.cat(pillars, dim=0) # (p1 + p2 + ... + pb, num_points, c)
        npoints_per_pillar = torch.cat(npoints_per_pillar, dim=0) # (p1 + p2 + ... + pb, )
        coors_batch = []
        for i, cur_coors in enumerate(coors):
            coors_batch.append(F.pad(cur_coors, (1, 0), value=i))
        coors_batch = torch.cat(coors_batch, dim=0) # (p1 + p2 + ... + pb, 1 + 3)

        return pillars, coors_batch, npoints_per_pillar


In [6]:
pc_folder_path = "dataset/testing/velodyne/"
pc_file_list = os.listdir(pc_folder_path)
pc = read_points(pc_folder_path + "/" + pc_file_list[0])

pc = point_range_filter(pc)
# pc_torch = torch.from_numpy(pc)

NameError: name 'point_range_filter' is not defined

In [7]:
if __name__ == "__main__":

    # np.random.seed(22)

    device = torch.device("cuda:0")

    nclasses=3
    voxel_size=[0.16, 0.16, 4]
    point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1]
    max_num_points=32
    max_voxels=(16000, 40000)

    test_layer = PillarLayer(voxel_size=voxel_size,
                            point_cloud_range=point_cloud_range,
                            max_voxels=max_voxels,
                            max_num_points=max_num_points)

    # pc = np.random.uniform(-2, 8, size=[1000, 4]).astype(np.float32)
    pc_tensor = torch.from_numpy(pc).to(device)

    pillars, coors_batch, npoints_per_pillar = test_layer([pc_tensor])

    print(pillars.size())
    print(coors_batch.size())
    print(npoints_per_pillar.size())

    print(pillars[10])
    print(coors_batch[10])
    print(npoints_per_pillar[10])



torch.Size([8599, 32, 4])
torch.Size([8599, 4])
torch.Size([8599])
tensor([[20.3780,  4.6160,  0.9160,  0.3900],
        [20.4490,  4.5740,  0.7840,  0.2900],
        [20.3510,  4.6190,  0.7820,  0.3900],
        [20.4170,  4.5080,  0.4190,  0.2600],
        [20.3700,  4.5640,  0.4180,  0.3900],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0

In [51]:
vx, vy = voxel_size[0], voxel_size[1]
x_offset = voxel_size[0] / 2 + point_cloud_range[0]
y_offset = voxel_size[1] / 2 + point_cloud_range[1]

offset_pt_center = pillars[:, :, :3] - torch.sum(pillars[:, :, :3], dim=1, keepdim=True) / npoints_per_pillar[:, None, None] 
x_offset_pi_center = pillars[:, :, :1] - (coors_batch[:, None, 1:2] * vx + x_offset) # (p1 + p2 + ... + pb, num_points, 1)
y_offset_pi_center = pillars[:, :, 1:2] - (coors_batch[:, None, 2:3] * vy + y_offset) # (p1 + p2 + ... + pb, num_points, 1)


features = torch.cat([pillars, offset_pt_center, x_offset_pi_center, y_offset_pi_center], dim=-1) # (p1 + p2 + ... + pb, num_points, 9)
features[:, :, 0:1] = x_offset_pi_center # tmp
features[:, :, 1:2] = y_offset_pi_center # tmp

# voxel_ids = torch.arange(0, pillars.size(1)).to(device) # (num_points, )
# mask = voxel_ids[:, None] < npoints_per_pillar[None, :] # (num_points, p1 + p2 + ... + pb)
# mask = mask.permute(1, 0).contiguous()  # (p1 + p2 + ... + pb, num_points)
# features *= mask[:, :, None]




In [52]:
conv = nn.Conv1d(9, 64, 1, bias=False).to(device)
bn = nn.BatchNorm1d(64, eps=1e-3, momentum=0.01).to(device)
# 5. embedding
features = features.permute(0, 2, 1).contiguous()
features = F.relu(bn(conv(features)))  # (p1 + p2 + ... + pb, out_channels, num_points)
pooling_features = torch.max(features, dim=-1)[0] # (p1 + p2 + ... + pb, out_channels)




In [53]:
pooling_features.size()

torch.Size([8599, 64])

In [89]:
batched_canvas = []
bs = coors_batch[-1, 0] + 1

x_l = int((point_cloud_range[3] - point_cloud_range[0]) / voxel_size[0])
y_l = int((point_cloud_range[4] - point_cloud_range[1]) / voxel_size[1])

for i in range(bs):
    cur_coors_idx = coors_batch[:, 0] == i
    cur_coors = coors_batch[cur_coors_idx, :]
    cur_features = pooling_features[cur_coors_idx]
    canvas = torch.zeros((x_l, y_l, 64), dtype=torch.float32, device=device)
    canvas[cur_coors[:, 1], cur_coors[:, 2]] = cur_features
    canvas = canvas.permute(2, 1, 0).contiguous()
    batched_canvas.append(canvas)
batched_canvas = torch.stack(batched_canvas, dim=0)

In [78]:
in_channel=64
out_channels=[64, 128, 256]
layer_nums=[3, 5, 5]
layer_strides=[2, 2, 2]

for i in range(len(layer_strides)):
    blocks = []
    blocks.append(nn.Conv2d(in_channel, out_channels[i], 3, stride=layer_strides[i], bias=False, padding=1))
    blocks.append(nn.BatchNorm2d(out_channels[i], eps=1e-3, momentum=0.01))
    blocks.append(nn.ReLU(inplace=True))

    for _ in range(layer_nums[i]):
        blocks.append(nn.Conv2d(out_channels[i], out_channels[i], 3, bias=False, padding=1))
        blocks.append(nn.BatchNorm2d(out_channels[i], eps=1e-3, momentum=0.01))
        blocks.append(nn.ReLU(inplace=True))

In [103]:
a1 = nn.Conv2d(64, 64, 3, stride=2, bias=False, padding=1).to(device)
a13 = nn.Conv2d(64, 64, 3, stride=1, bias=False).to(device)
a14 = nn.Conv2d(64, 64, 3, stride=1, bias=False).to(device)
a15 = nn.Conv2d(64, 64, 3, stride=1, bias=False).to(device)

b1 = nn.ConvTranspose2d(64, 128, 1, stride=1, bias=False).to(device)

c1


In [105]:
test = a1(batched_canvas)
test = b1(test)

# test = a12(batched_canvas)

In [107]:
torch.linspace(0, 40, 216 + 1)

tensor([ 0.0000,  0.1852,  0.3704,  0.5556,  0.7407,  0.9259,  1.1111,  1.2963,
         1.4815,  1.6667,  1.8519,  2.0370,  2.2222,  2.4074,  2.5926,  2.7778,
         2.9630,  3.1481,  3.3333,  3.5185,  3.7037,  3.8889,  4.0741,  4.2593,
         4.4444,  4.6296,  4.8148,  5.0000,  5.1852,  5.3704,  5.5556,  5.7407,
         5.9259,  6.1111,  6.2963,  6.4815,  6.6667,  6.8519,  7.0370,  7.2222,
         7.4074,  7.5926,  7.7778,  7.9630,  8.1481,  8.3333,  8.5185,  8.7037,
         8.8889,  9.0741,  9.2593,  9.4444,  9.6296,  9.8148, 10.0000, 10.1852,
        10.3704, 10.5556, 10.7407, 10.9259, 11.1111, 11.2963, 11.4815, 11.6667,
        11.8519, 12.0370, 12.2222, 12.4074, 12.5926, 12.7778, 12.9630, 13.1481,
        13.3333, 13.5185, 13.7037, 13.8889, 14.0741, 14.2593, 14.4444, 14.6296,
        14.8148, 15.0000, 15.1852, 15.3704, 15.5556, 15.7407, 15.9259, 16.1111,
        16.2963, 16.4815, 16.6667, 16.8519, 17.0370, 17.2222, 17.4074, 17.5926,
        17.7778, 17.9630, 18.1481, 18.33

In [59]:
ranges = [[0, -39.68, -0.6, 69.12, 39.68, -0.6],
            [0, -39.68, -0.6, 69.12, 39.68, -0.6],
            [0, -39.68, -1.78, 69.12, 39.68, -1.78]]
# ranges = [[0, -39.68, 0.6, 69.12, 39.68, 0.6],
#             [0, -39.68, 0.6, 69.12, 39.68, 0.6],
#             [0, -39.68, 1.78, 69.12, 39.68, 1.78]]
sizes = [[0.6, 0.8, 1.73], [0.6, 1.76, 1.73], [1.6, 3.9, 1.56]]
rotations=[0, 1.57]




rotations = torch.tensor(rotations, device=device)
# anchor_range = ranges[0]
anchor_range = [0, 0, -0.6, 10., 10., -0.6]
anchor_size = torch.tensor([0.6, 0.8, 1.73],device=device)

feature_map_size = (3,3)

x_centers = torch.linspace(anchor_range[0], anchor_range[3], feature_map_size[1] + 1, device=device)
y_centers = torch.linspace(anchor_range[1], anchor_range[4], feature_map_size[0] + 1, device=device)
z_centers = torch.linspace(anchor_range[2], anchor_range[5], 1 + 1, device=device)


x_shift = (x_centers[1] - x_centers[0]) / 2
y_shift = (y_centers[1] - y_centers[0]) / 2
z_shift = (z_centers[1] - z_centers[0]) / 2
x_centers = x_centers[:feature_map_size[1]] + x_shift # (feature_map_size[1], )
y_centers = y_centers[:feature_map_size[0]] + y_shift # (feature_map_size[0], )
z_centers = z_centers[:1] + z_shift  # (1, )

meshgrids = torch.meshgrid(x_centers, y_centers, z_centers, rotations)
meshgrids = list(meshgrids)
for i in range(len(meshgrids)):
    meshgrids[i] = meshgrids[i][..., None] # [feature_map_size[1], feature_map_size[0], 1, 2, 1]

anchor_size = anchor_size[None, None, None, None, :]
repeat_shape = [feature_map_size[1], feature_map_size[0], 1, len(rotations), 1]
anchor_size = anchor_size.repeat(repeat_shape) # [feature_map_size[1], feature_map_size[0], 1, 2, 3]
meshgrids.insert(3, anchor_size)
anchors = torch.cat(meshgrids, dim=-1).permute(2, 1, 0, 3, 4).contiguous()


In [60]:
anchor_size.size()

torch.Size([3, 3, 1, 2, 3])

In [61]:
anchors.size()

torch.Size([1, 3, 3, 2, 7])